In [ ]:
from graph_tool.all import graph_draw 

from osmgt import OsmGt


# build the graph
graph = OsmGt.get_graph_from_location("Roanne")

pos = draw.sfdp_layout(graph)
graph_draw(graph, pos=pos, output="output.svg")